In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

c:\Users\vidyu\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-03 13:25:16,268 INFO: Initializing external client
2025-03-03 13:25:16,268 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 13:25:17,553 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215639
Fetching data from 2025-02-02 18:25:16.261955+00:00 to 2025-03-03 17:25:16.261955+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.38s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-02 19:00:00+00:00,2,0
1,2025-02-02 20:00:00+00:00,2,0
2,2025-02-02 21:00:00+00:00,2,0
3,2025-02-02 22:00:00+00:00,2,0
4,2025-02-02 23:00:00+00:00,2,0
...,...,...,...
170675,2025-03-02 22:00:00+00:00,263,45
170676,2025-03-02 23:00:00+00:00,263,17
170677,2025-03-03 00:00:00+00:00,263,12
170678,2025-03-03 01:00:00+00:00,263,13


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 170680 entries, 0 to 176703
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         170680 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  170680 non-null  int32                  
 2   rides               170680 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 3.9 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 178962
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174445 non-null  datetime64[us]
 1   pickup_location_id  174445 non-null  int32         
 2   rides               174445 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [8]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [9]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,0,0,0,3,0,0,3,5,1,...,1,1,0,1,0,6,1,9,255,2025-03-02 19:00:00
1,2,1,0,1,1,1,0,1,1,1,...,0,1,0,1,1,0,1,0,218,2025-03-02 19:00:00
2,369,273,249,21,251,348,17,373,286,251,...,215,283,315,306,310,328,323,263,237,2025-03-02 19:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,57,2025-03-02 19:00:00
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,56,2025-03-02 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,187,2025-03-02 19:00:00
247,0,2,0,0,4,0,1,0,0,2,...,1,3,0,0,0,1,0,0,71,2025-03-02 19:00:00
248,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,8,2025-03-02 19:00:00
249,0,1,1,1,2,0,4,0,1,1,...,0,2,1,2,4,4,2,3,168,2025-03-02 19:00:00


In [10]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-03 13:25:42,409 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 13:25:42,414 INFO: Initializing external client
2025-03-03 13:25:42,414 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 13:25:43,481 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215639
Fetching data from 2025-02-02 18:25:42.409917+00:00 to 2025-03-03 17:25:42.409917+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (66.77s) 


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
current_date

Timestamp('2025-03-02 21:46:04.416865+0000', tz='Etc/UTC')

In [12]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-02 19:00:00
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,3,2025-03-02 19:00:00
2,1,0,4,1,0,1,0,1,0,1,...,3,5,6,4,2,7,6,8,4,2025-03-02 19:00:00
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,6,2025-03-02 19:00:00
4,3,1,2,1,3,2,0,0,0,1,...,1,2,1,2,2,2,4,1,7,2025-03-02 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,259,2025-03-02 19:00:00
247,0,2,0,0,1,1,2,1,0,1,...,2,1,1,1,2,1,2,3,260,2025-03-02 19:00:00
248,25,9,5,14,5,2,0,0,1,0,...,23,38,36,41,58,49,42,35,261,2025-03-02 19:00:00
249,32,24,22,10,7,2,2,2,0,7,...,121,89,87,87,68,72,65,72,262,2025-03-02 19:00:00


In [11]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-03 13:27:02,763 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 13:27:02,763 INFO: Initializing external client
2025-03-03 13:27:02,763 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 13:27:03,965 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215639


In [13]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [14]:
predictions

,pickup_location_id,predicted_demand
0,2,-0.0
1,3,0.0
2,4,7.0
3,6,0.0
4,7,3.0
...,...,...
246,259,-0.0
247,260,1.0
248,261,26.0
249,262,48.0


In [15]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 138, 236, 186, 142, 237, 161, 239, 249,  68])